## LOFAR VIDEO MAKER

In [1]:
%load_ext autoreload
%autoreload 2

# Imports

In [2]:
import os
import sys
import time
import datetime
import glob
import numpy as np

from lofarantpos.db import LofarAntennaDatabase
from lofarimaging import read_acm_cube, get_station_type, make_xst_plots
from lofarantpos.db import LofarAntennaDatabase

# Data folders

In [3]:
caltable_dir = "../image_data/CalTables/"
data_files = "../image_data/sess_sid20230111T071252_LV614/scan_59955.30061/LV614_20230111_071256_spw3_sb51_461_int1_dur24300_dir0,0,SUN_xst/"
os.makedirs('results', exist_ok=True)

# General Values

In [4]:
station_name = "LV614"
rcu_mode = "3"
station_type = get_station_type(station_name)

In [5]:
import re
 # Read subband from .h file
def get_subbands(file_path):

    with open(file_path, 'r') as file:
        content = file.read()

    # Find the subband number using regex
    match = re.search(r'--xcsubband=(\d+)', content)
    if match:
        subband = int(match.group(1))
    else:
        subband = None

    return subband

def get_visibilitiesObstime(file_path):
    obsdatestr, obstimestr, *_ = os.path.basename(file_path).rstrip(".dat").split("_")
    obstime = datetime.datetime.strptime(obsdatestr + ":" + obstimestr, '%Y%m%d:%H%M%S')
    visibilities = read_acm_cube(file_path, station_type)[0]
    return visibilities, obstime

dat_files = sorted(glob.glob(os.path.join(data_files, '*.dat')))[:100]
h_files = sorted(glob.glob(os.path.join(data_files, '*.h')))[:100]

assert len(dat_files) == len(h_files), "Mismatch in number of .dat and .h files"

# Iterate through each pair of .dat and .h files
for dat_file, h_file in zip(dat_files, h_files):
    # Process the .h file
    subband = get_subbands(h_file)
    print("Subband: ", subband)
    visibilities, obstime = get_visibilitiesObstime(dat_file)
    # Process the .dat file
    sky_fig, ground_fig, leaflet_map = make_xst_plots(visibilities, station_name, obstime, subband, rcu_mode, outputpath="./video")



Subband:  51
Subband:  52
Subband:  53
Subband:  54
Subband:  55
No calibration table found... cube remains uncalibrated!


        Use get_body("moon") instead. [lofarimaging.singlestationutil]


Maximum at 148m east, -10m north of station center (lat/long 57.55679, 21.85739)
Subband:  56
No calibration table found... cube remains uncalibrated!


        Use get_body("moon") instead. [lofarimaging.singlestationutil]


Maximum at -150m east, -48m north of station center (lat/long 57.55645, 21.85241)
Subband:  57
No calibration table found... cube remains uncalibrated!
Maximum at -68m east, -150m north of station center (lat/long 57.55553, 21.85378)
Subband:  58
No calibration table found... cube remains uncalibrated!
Maximum at 148m east, -8m north of station center (lat/long 57.55681, 21.85739)
Subband:  59
No calibration table found... cube remains uncalibrated!
Maximum at 148m east, -20m north of station center (lat/long 57.55670, 21.85739)
Subband:  60
No calibration table found... cube remains uncalibrated!
Maximum at 32m east, -150m north of station center (lat/long 57.55553, 21.85545)
Subband:  61
No calibration table found... cube remains uncalibrated!
Maximum at 32m east, -150m north of station center (lat/long 57.55553, 21.85545)
Subband:  62
No calibration table found... cube remains uncalibrated!
Maximum at 114m east, 148m north of station center (lat/long 57.55821, 21.85682)
Subband:  63

ConnectionError: HTTPConnectionPool(host='server.arcgisonline.com', port=80): Max retries exceeded with url: /arcgis/rest/services/World_Imagery/MapServer/WMTS/1.0.0/WMTSCapabilities.xml?service=WMTS&request=GetCapabilities&version=1.0.0 (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x72f02b2c3680>: Failed to resolve 'server.arcgisonline.com' ([Errno -3] Temporary failure in name resolution)"))

In [ ]:
import cv2
import glob
import os

image_folder = "./video"

# image_files = sorted(glob.glob(os.path.join(image_folder, "*sky_calibrated*.png")))
image_files = sorted(glob.glob(os.path.join(image_folder, "*nearfield_calibrated*.png")))

if not image_files:
    print("No images found!")
    exit()

frame = cv2.imread(image_files[0])
height, width, layers = frame.shape

output_file = "nearfield_calibratedTest1.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
fps = 10  # Frames per second

video = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

for img in image_files:
    frame = cv2.imread(img)
    video.write(frame)

video.release()
print(f"Video saved as {output_file}")


Video saved as output.mp4
